In [ ]:
pip install transformers


In [ ]:
pip install nltk

In [ ]:
import re
import nltk
import numpy as np
import math
from math import log
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Initialize NLTK's Porter Stemmer
stemmer = PorterStemmer()

# Initialize the text generation pipeline with the desired model
generator = pipeline('text-generation', model='gpt2')

# Define the prompts
topics = [
    "machine learning",
    "Lionel Messi"
]

# List to store generated documents
generated_documents = []

# Generate text based on each topic and store the documents
for i, topic in enumerate(topics, start=1):
    generated_text = generator(topic, max_length=50)
    generated_documents.append(generated_text[0]['generated_text'])

# Print the generated documents
print("Generated Documents:")
for i, doc in enumerate(generated_documents, start=1):
    print("Document", i, ":", doc)
    print()

# Initialize NLTK's stopwords
stop_words = set(stopwords.words('english'))

# Define a function to clean and normalize the text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove symbols and characters
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    # Split the text into words
    words = cleaned_text.split()
    # Remove stop words and stem each word
    filtered_words = [stemmer.stem(word) for word in words if word not in stop_words]
    return filtered_words

# List to store unique words for each document
preprocessed_documents = []

# Get unique words from each document separately
for doc in generated_documents:
    preprocessed_doc = preprocess_text(doc)
    unique_words = set(preprocessed_doc)
    preprocessed_documents.append(unique_words)

# Print the unique words for each document separately
for i, unique_words in enumerate(preprocessed_documents, start=1):
    print("Unique words in Document", i, ":")
    for word in unique_words:
        print(word)
    print()

# Compute TF
def compute_tf(document):
    tf = Counter(document)
    for word in tf:
        tf[word] = tf[word] / len(document)
    return tf

# Compute IDF
def compute_idf(documents):
    idf = {}
    total_docs = len(documents)
    words_in_documents = [set(document) for document in documents]
    all_words = set().union(*words_in_documents)
    for word in all_words:
        doc_count = sum([1 for document in documents if word in document])
        idf[word] = log(total_docs / (doc_count + 1))
    return idf

# Compute TF-IDF
def compute_tfidf(documents):
    tfidf = []
    idf = compute_idf(documents)
    for document in documents:
        tf = compute_tf(document)
        tfidf_doc = {word: tf[word] * idf[word] for word in tf}
        tfidf.append(tfidf_doc)
    return tfidf

# Compute TF-IDF from scratch
tfidf_scratch = compute_tfidf(preprocessed_documents)

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Preprocess the documents
preprocessed_documents = [' '.join(doc) for doc in preprocessed_documents]

# Compute TF-IDF using scikit-learn
tfidf_matrix = vectorizer.fit_transform(preprocessed_documents)
feature_names = vectorizer.get_feature_names_out()

# Print TF-IDF using scikit-learn
print("TF-IDF using scikit-learn:")
for i in range(len(preprocessed_documents)):
    print("Document", i+1, ":")
    for j, word in enumerate(feature_names):
        tfidf_value = tfidf_matrix[i, j]
        if tfidf_value != 0:
            print(word, ":", tfidf_value)
    print()